# Services

The second type of connections between nodes is a service.

Nodes can run a server, which only serves data when it is requested to by a client. It's similar to a regular function in programming, except it works across nodes.

Let's make a server that does the same thing as in our publisher/subscriber node, but only when requested. When you give the server a String, it will return a string with some punctuation added to the end.

The first thing we have to do is define the data types, for input and output. Our inputs and outputs are both strings, so lets define a message that consists of an input and output string.

## Messages

The way we can define what goes in our message is with `msg` and `srv` files. Message definitions must be in a CMake package and not a python package, so we'll define our messages in another package, but we can use them anywhere as long as our workspace is loaded. Run the following commands to create a new package.

In [ ]:
# Initialize your workspace variables
workspace = '/home/ubuntu/learn_ros_ws'
%cd $workspace
package_name = 'basic_nodes'
interface_package_name = 'learn_ros_interfaces'

In [ ]:
%%bash --no-raise-error -s "$package_name" "$interface_package_name"
cd src

# Create a package
# ros2 pkg create --build-type ament_python $1 # Should already be created. Uncomment if you skipped the first tutorial.
ros2 pkg create --build-type ament_cmake $2

Now, we need to create two directories in our package: `msg` and `srv`. This is where our message and service definitions will go.

In [ ]:
%%bash --no-raise-error -s "$interface_package_name"

mkdir src/$1/src/msg src/$1/src/srv

So, what goes in these directiories? We need our message and service files.

The name of the file is the name of your message or service, and the contents are defined as follows:

```
type1 part1
type2 part2
```

This is like a struct in programming.

Service definitions are similar, and are defined as follows:

```
type1 input1
type2 input2
---
type3 output
```

A service actually consists of two messages: One above the line, and one below it. This means that you can have as many inputs or outputs as you want!

For our service, lets define a service with one input string and one output string.

In [ ]:
%%writefile src/$interface_package_name/src/srv/AddPunctuation.srv

string input
---
string output

Now we need to tell our package about our new service. We're going to edit `package.xml` and `CMakeLists.txt`, which are two files that tell the build system what other files should be used.

Add these lines to the end of your CMakeLists.txt in the interfaces.

```cmake
find_package(rosidl_default_generators REQUIRED)

rosidl_generate_interfaces(${PROJECT_NAME}
  "srv/AddPunctuation.srv"
)
```

Note that we didn't actually make any custom messages, since we don't need them for this tutorial. If you did need to make a message, it would need to be added here as well.

Your final CMakeLists.txt should look like this:

In [ ]:
%%writefile src/$interface_package_name/CMakeLists.txt

cmake_minimum_required(VERSION 3.8)
project(learn_ros_interfaces)

if(CMAKE_COMPILER_IS_GNUCXX OR CMAKE_CXX_COMPILER_ID MATCHES "Clang")
  add_compile_options(-Wall -Wextra -Wpedantic)
endif()

# find dependencies
find_package(ament_cmake REQUIRED)
find_package(rosidl_default_generators REQUIRED)
# uncomment the following section in order to fill in
# further dependencies manually.
# find_package(<dependency> REQUIRED)

rosidl_generate_interfaces(${PROJECT_NAME}
  "src/srv/AddPunctuation.srv"
)

if(BUILD_TESTING)
  find_package(ament_lint_auto REQUIRED)
  # the following line skips the linter which checks for copyrights
  # comment the line when a copyright and license is added to all source files
  set(ament_cmake_copyright_FOUND TRUE)
  # the following line skips cpplint (only works in a git repo)
  # comment the line when this package is in a git repo and when
  # a copyright and license is added to all source files
  set(ament_cmake_cpplint_FOUND TRUE)
  ament_lint_auto_find_test_dependencies()
endif()

ament_package()

We're also going to need to edit `package.xml`, right next to `CMakeLists.txt`.

Add these lines to `package.xml`:

```
  <buildtool_depend>rosidl_default_generators</buildtool_depend>
  <exec_depend>rosidl_default_runtime</exec_depend>
  <member_of_group>rosidl_interface_packages</member_of_group>
```

So your final file should look like this:

In [ ]:
%%writefile src/$interface_package_name/package.xml
<?xml version="1.0"?>
<?xml-model href="http://download.ros.org/schema/package_format3.xsd" schematypens="http://www.w3.org/2001/XMLSchema"?>
<package format="3">
  <name>learn_ros_interfaces</name>
  <version>0.0.0</version>
  <description>TODO: Package description</description>
  <maintainer email="youremailhere@example.com">root</maintainer>
  <license>TODO: License declaration</license>

  <buildtool_depend>ament_cmake</buildtool_depend>

  <buildtool_depend>rosidl_default_generators</buildtool_depend>
  <exec_depend>rosidl_default_runtime</exec_depend>
  <member_of_group>rosidl_interface_packages</member_of_group>

  <test_depend>ament_lint_auto</test_depend>
  <test_depend>ament_lint_common</test_depend>

  <export>
    <build_type>ament_cmake</build_type>
  </export>
</package>


And now build! Once you reload your terminal, you should see your new service interface in the ros2 list.

In [ ]:
%%bash --no-raise-error -s "$interface_package_name"

colcon build --packages-select $1
source install/local_setup.bash
ros2 interface list | grep AddPunctuation

Your output should look like this:

```
Starting >>> learn_ros_interfaces
Finished <<< learn_ros_interfaces [0.45s]

Summary: 1 package finished [0.54s]
    learn_ros_interfaces/srv/AddPunctuation
```

If you don't see the last line, go back and make sure you followed all the steps.

# Writing a Server

To use make use of our service, we need to write a server! This will actually read the request message, generate the response, and send it back to the client.

In [ ]:
%%writefile src/$package_name/$package_name/server_node.py
import rclpy
from rclpy.node import Node
from learn_ros_interfaces.srv import AddPunctuation


class MyServer(Node):
    def __init__(self):
        super().__init__('server_node')
        
        self.my_server = self.create_service(AddPunctuation, 'boring_strings_srv', self.add_punctuation)
        
    def add_punctuation(self, request, response):
        response.output = request.input + '!'
        return response
    

def main(args=None):
    print('Starting server node...')
    rclpy.init(args=args)
    server = MyServer()
    rclpy.spin(server)


if __name__ == '__main__':
    main()

Don't forget to add the new entrypoint to your `setup.py`! 

```python
    entry_points={
        'console_scripts': [
            # other nodes,
            'server_node = basic_nodes.server_node:main'
        ]
    }
```

It's pretty similar to our subscriber from before! Notice the first argument of create_service; it's the `.srv` type we just created! We also had to import it from our other package, `learn_ros_interfaces`.

Once we build and spin up our server, we can send a request to our server with the following command:

```bash
ros2 run run basic_nodes server_node
```

In [ ]:
%%bash --no-raise-error -s "$package_name"
colcon build --packages-select $1
source install/local_setup.bash
ros2 run basic_nodes server_node

We can send a request to our server from the command line. Try it out!

```bash
ros2 service call /boring_strings_srv learn_ros_interfaces/srv/AddPunctuation "{input: Hello}"
```

You should get a response with some exciting punctuation added to the end of it!

As with the publisher, we don't want to call the service from the command line every time. Instead, we want to write another node that can call the service by itself. Let's write a client node to use our server node.

In [ ]:
%%writefile src/$package_name/$package_name/client_node.py

# https://docs.ros.org/en/jazzy/Tutorials/Beginner-Client-Libraries/Writing-A-Simple-Py-Service-And-Client.html#write-the-client-node

import sys
import rclpy
from rclpy.node import Node
from learn_ros_interfaces.srv import AddPunctuation


class MyClient(Node):
    def __init__(self):
        super().__init__('client_node')
        
        self.client = self.create_client(AddPunctuation, 'boring_strings_cli')
        while not self.cli.wait_for_service(timeout_sec=1.0):
            self.get_logger().info('service not available, waiting again...')
        self.req = AddPunctuation.Request()
        
    def send_request(self, input):
        self.req.input = input
        return self.client.call_async(self.req)
        
    

def main(args=None):
    print('Starting client node...')
    rclpy.init(args=args)
    client = MyClient()
    future = client.send_request(sys.argv[1])
    rclpy.spin_until_future_complete(client, future)
    response = future.result()
    client.get_logger().info('Boring string:\t' + sys.argv[1] +
                             '\nExciting String:\t' + response.output)
    
    client.destroy_node()
    rclpy.shutdown()


if __name__ == '__main__':
    main()

Notice that this main function is a little more complicated than the other ones. That's because after the client finishes, it needs to shut itself down. It can't just run forever. Most of the code in the function telling the client node that once it finishes its task, it should stop and shut down.

Update your `setup.py again`:

```python
    entry_points={
        'console_scripts': [
            # Other nodes
            'server_node = basic_nodes.server_node:main',
            'client_node = basic_nodes.client_node:main'
        ],
    },
```

And run your `client_node`! Remember to build and re-source. 

Note that it takes in a commandline parameter, so make sure you call it with one.

```bash
ros2 run basic_nodes client_node "Hello from the client"
```

In [ ]:
%%bash --no-raise-error -s "$package_name"

colcon build --packages-select $1

Now you've built a server and client node! In the next section, you'll make your own server and client to... TODO!